In [1]:
import numpy as np
import pandas as pd
import time
import pickle


pd.options.display.max_rows = 2000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 255

In [8]:
# results_df = None

with open('./pickles/latest_results.pkl', 'rb') as f:
    results_df = pickle.load(f)
    
    

In [7]:
def choose_largest_coefficient_name (coefficients):
    if coefficients is None:
        return None
    return coefficients.sort_values(by='abs_weight', ascending=False)['X_Column'].iloc[0]

def choose_largest_coefficient_weight (coefficients):    
    if coefficients is None:
        return None
    return coefficients.sort_values(by='abs_weight', ascending=False)['abs_weight'].iloc[0]
    
    
results_df['largest_coefficient_column'] = results_df['coefficients'].apply(choose_largest_coefficient_name)
results_df['largest_coefficient_weight'] = results_df['coefficients'].apply(choose_largest_coefficient_weight)

results_df \
    .drop(columns=['coefficients', 'X_columns']) \
    .query('TP > 0 or FP > 0')

P     N    TP    FP    TN    FN    TPR    TNR  \
country        cutoff                                                     
United States  5000    1882  7787   373   208  7579  1509  0.198  0.973   
               5000    1882  7787   297    39  7748  1585  0.158  0.995   
               5000    1882  7787   224   235  7552  1658  0.119  0.970   
               2500    1882  7787   444   232  7555  1438  0.236  0.970   
               2500    1882  7787   295    32  7755  1587  0.157  0.996   
               2500    1882  7787   119   180  7607  1763  0.063  0.977   
               1000    1882  7787   504   252  7535  1378  0.268  0.968   
               1000    1882  7787   206    25  7762  1676  0.109  0.997   
               1000    1882  7787    57    60  7727  1825  0.030  0.992   
               500     1882  7787   523   254  7533  1359  0.278  0.967   
               500     1882  7787   183    21  7766  1699  0.097  0.997   
               500     1882  7787   436   581  7206  1446  0.232  0.925   
               250     1882  7787   541   262  7525  1341  0.287  0.966   
               250     1882  7787   236    29  7758  1646  0.125  0.996   
               250     1882  7787  1621  4788  2999   261  0.861  0.385   
               100     1882  7787   545   264  7523  1337  0.290  0.966   
               100     1882  7787   143    20  7767  1739  0.076  0.997   
               100     1882  7787   721  1342  6445  1161  0.383  0.828   
India          5000    1285  8384   180    32  8352  1105  0.140  0.996   
               5000    1285  8384    30     7  8377  1255  0.023  0.999   
               5000    1285  8384     1    59  8325  1284  0.001  0.993   
               2500    1285  8384   249    59  8325  1036  0.194  0.993   
               2500    1285  8384    19     2  8382  1266  0.015  1.000   
               2500    1285  8384  1022  6406  1978   263  0.795  0.236   
               1000    1285  8384   514   224  8160   771  0.400  0.973   
               1000    1285  8384    36    10  8374  1249  0.028  0.999   
               1000    1285  8384   642  3381  5003   643  0.500  0.597   
               500     1285  8384   520   219  8165   765  0.405  0.974   
               500     1285  8384    32     7  8377  1253  0.025  0.999   
               500     1285  8384   842  4714  3670   443  0.655  0.438   
               250     1285  8384   519   216  8168   766  0.404  0.974   
               250     1285  8384    69    13  8371  1216  0.054  0.998   
               100     1285  8384   518   217  8167   767  0.403  0.974   
               100     1285  8384    54    12  8372  1231  0.042  0.999   
               100     1285  8384     1     3  8381  1284  0.001  1.000   
Germany        5000     608  9061    14   346  8715   594  0.023  0.962   
               1000     608  9061    11   196  8865   597  0.018  0.978   
United Kingdom 5000     594  9075    51   230  8845   543  0.086  0.975   
               2500     594  9075    37   154  8921   557  0.062  0.983   
               500      594  9075    37   157  8918   557  0.062  0.983   
               250      594  9075    19   113  8962   575  0.032  0.988   
Canada         5000     309  9360    11   201  9159   298  0.036  0.979   
               2500     309  9360     0    42  9318   309  0.000  0.996   
               250      309  9360   127  3346  6014   182  0.411  0.643   

                         PPV    NPV    ACC         model_type  column_count  \
country        cutoff                                                         
United States  5000    0.642  0.834  0.822  linear_regression           193   
               5000    0.884  0.830  0.832      random_forest           193   
               5000    0.488  0.820  0.804         linear_svm           193   
               2500    0.657  0.840  0.827  linear_regression           291   
               2500    0.902  0.830  0.833      random_forest           291   
               2500   

In [9]:
def choose_largest_coefficient_name (coefficients):
    if coefficients is None:
        return None
    return coefficients.sort_values(by='abs_weight', ascending=False)['X_Column'].iloc[0]

def choose_largest_coefficient_weight (coefficients):    
    if coefficients is None:
        return None
    return coefficients.sort_values(by='abs_weight', ascending=False)['abs_weight'].iloc[0]
    
    
results_df['largest_coefficient_column'] = results_df['coefficients'].apply(choose_largest_coefficient_name)
results_df['largest_coefficient_weight'] = results_df['coefficients'].apply(choose_largest_coefficient_weight)

results_df \
    .drop(columns=['coefficients', 'X_columns']) \
    .query('TP > 0 or FP > 0')

P      N    TP     FP     TN    FN    TPR    TNR  \
country        cutoff                                                        
United States  5000    3734  15605   898    394  15211  2836  0.240  0.975   
               5000    3734  15605   724    105  15500  3010  0.194  0.993   
               5000    3734  15605   304    817  14788  3430  0.081  0.948   
               2500    3734  15605   963    402  15203  2771  0.258  0.974   
               2500    3734  15605   602     89  15516  3132  0.161  0.994   
               2500    3734  15605   109    486  15119  3625  0.029  0.969   
               1000    3734  15605  1019    416  15189  2715  0.273  0.973   
               1000    3734  15605   533     70  15535  3201  0.143  0.996   
               1000    3734  15605    29    118  15487  3705  0.008  0.992   
               500     3734  15605  1030    424  15181  2704  0.276  0.973   
               500     3734  15605   548     73  15532  3186  0.147  0.995   
               500     3734  15605  1662   3238  12367  2072  0.445  0.793   
               250     3734  15605  1036    426  15179  2698  0.277  0.973   
               250     3734  15605   450     50  15555  3284  0.121  0.997   
               250     3734  15605   376    847  14758  3358  0.101  0.946   
               100     3734  15605  1035    426  15179  2699  0.277  0.973   
               100     3734  15605   552     68  15537  3182  0.148  0.996   
               100     3734  15605   729   1244  14361  3005  0.195  0.920   
India          5000    2560  16779   522    115  16664  2038  0.204  0.993   
               5000    2560  16779    96     11  16768  2464  0.037  0.999   
               5000    2560  16779     0      1  16778  2560  0.000  1.000   
               2500    2560  16779  1044    408  16371  1516  0.408  0.976   
               2500    2560  16779   200     31  16748  2360  0.078  0.998   
               2500    2560  16779  2086  13223   3556   474  0.815  0.212   
               1000    2560  16779  1061    418  16361  1499  0.414  0.975   
               1000    2560  16779   231     37  16742  2329  0.090  0.998   
               1000    2560  16779   885   4588  12191  1675  0.346  0.727   
               500     2560  16779  1055    419  16360  1505  0.412  0.975   
               500     2560  16779   224     36  16743  2336  0.087  0.998   
               500     2560  16779  1925  11013   5766   635  0.752  0.344   
               250     2560  16779  1054    417  16362  1506  0.412  0.975   
               250     2560  16779   131     24  16755  2429  0.051  0.999   
               250     2560  16779   396   1599  15180  2164  0.155  0.905   
               100     2560  16779  1054    419  16360  1506  0.412  0.975   
               100     2560  16779   181     24  16755  2379  0.071  0.999   
               100     2560  16779     1    188  16591  2559  0.000  0.989   
United Kingdom 5000    1131  18208     0      1  18207  1131  0.000  1.000   
               1000    1131  18208     0     31  18177  1131  0.000  0.998   
               100     1131  18208    15    142  18066  1116  0.013  0.992   
Germany        500     1158  18181     0      1  18180  1158  0.000  1.000   
               250     1158  18181    18    538  17643  1140  0.016  0.970   
               100     1158  18181     0      1  18180  1158  0.000  1.000   
Canada         5000     665  18674    22    469  18205   643  0.033  0.975   
               1000     665  18674    19    315  18359   646  0.029  0.983   
               500      665  18674    22    399  18275   643  0.033  0.979   
               250      665  18674     1     64  18610   664  0.002  0.997   
               100      665  18674     8    149  18525   657  0.012  0.992   

                         PPV    NPV    ACC         model_type  column_count  \
country        cutoff                                                         
United States  5000    0.695  0.843  0.833  l